In [1]:
import random
import numpy as np

class GATimetable:
    def __init__(self, tasks, population_size, mutation_rate=0.3):
        self.tasks = tasks    #les modules
        self.population_size = population_size #nombre de tableau 
        self.mutation_rate = mutation_rate
        self.population = []

    def create_random_chromosome(self):
        # Create a random timetable chromosome
        timetable = []
        for task in self.tasks:
            day = random.randint(1, 5)  # Random day (Monday to Friday)
            start_time = random.randint(8, 14)  # Random start time (8 to 16)
            end_time = start_time + task.duration  # Calculate end time
            timetable.append((task, day, start_time, end_time))
        return timetable

    def initialize_population(self):
        # Create an initial population of random timetables
        self.population = [self.create_random_chromosome() for _ in range(self.population_size)]

    def calculate_overlap(self, timetable):
        # Calculate the overlap score for a given timetable
        overlap = 0
        for i in range(len(timetable)):
            for j in range(i + 1, len(timetable)):
                _, day1, start_time1, end_time1 = timetable[i]
                _, day2, start_time2, end_time2 = timetable[j]
                if day1 == day2 and end_time1 > start_time2 and end_time2 > start_time1:
                    overlap += 1
        return overlap

    def calculate_fitness(self):
        # Calculate the fitness score for each timetable in the population
        fitness_scores = []
        for timetable in self.population:
            overlap = self.calculate_overlap(timetable)
            fitness_scores.append(1.0 / (1.0 + overlap))  # Inverse of overlap
        return fitness_scores

    def selection(self, fitness_scores):
        # Select parents from the population using fitness-based selection
        total_fitness = sum(fitness_scores)
        probabilities = [fitness / total_fitness for fitness in fitness_scores]

        selected_parents = []
        for _ in range(self.population_size):
            parent_idx = np.random.choice(len(self.population), p=probabilities)
            selected_parents.append(self.population[parent_idx])

        return selected_parents


    def crossover(self, parents):
        # Perform crossover between parents to generate offspring
        offspring = []
        for i in range(0, len(parents), 2):
            parent1 = parents[i]
            parent2 = parents[i + 1]
            child = []
            for j in range(len(parent1)):
                if random.random() < 0.5:
                    child.append(parent1[j])
                else:
                    child.append(parent2[j])
            offspring.append(child)
        return offspring

    def mutation(self, offspring):
        # Perform mutation on the offspring
        for i in range(len(offspring)):
            if random.random() < self.mutation_rate:
                task_idx = random.randint(0, len(self.tasks) - 1)
                task = self.tasks[task_idx]
                day = random.randint(1, 5)
                start_time = random.randint(8, 14)
                end_time = start_time + task.duration
                offspring[i][task_idx] = (task, day, start_time, end_time)
        return offspring

    def generate_timetable(self, num_generations):
        self.initialize_population()

        for generation in range(num_generations):
            fitness_scores = self.calculate_fitness()
            parents = self.selection(fitness_scores)
            offspring = self.crossover(parents)
            offspring = self.mutation(offspring)
            parents.extend(offspring)
            self.population = parents

        fitness_scores = self.calculate_fitness()
        best_timetable_idx = np.argmax(fitness_scores)
        best_timetable = self.population[best_timetable_idx]

        return best_timetable

In [2]:
class Task:
    def __init__(self, name, duration):
        self.name = name
        self.duration = duration

# Define tasks
tasks = [
    Task("Math", 2),
    Task("English", 3),
    Task("Arabic", 4),
    Task("TEC", 2),
    Task("DEV", 3)
]

ga_timetable = GATimetable(tasks, population_size=50, mutation_rate=0.3)

best_timetable = ga_timetable.generate_timetable(num_generations=100)

for task, day, start_time, end_time in best_timetable:
    print(f"Task: {task.name}, Day: {day}, Start Time: {start_time}, End Time: {end_time}")

Task: Math, Day: 4, Start Time: 8, End Time: 10
Task: English, Day: 2, Start Time: 8, End Time: 11
Task: Arabic, Day: 5, Start Time: 10, End Time: 14
Task: TEC, Day: 1, Start Time: 11, End Time: 13
Task: DEV, Day: 3, Start Time: 13, End Time: 16
